In [3]:
"""Agents: auto plan agents example?

    Examples:

        Execute the following command in the terminal:
        Set env params.
        .. code-block:: shell

            export OPENAI_API_KEY=sk-xx
            export OPENAI_API_BASE=https://xx:80/v1

        run example.
        ..code-block:: shell
            python examples/agents/auto_plan_agent_dialogue_example.py
"""

import os
from dbgpt.agent.agents.user_proxy_agent import UserProxyAgent
from dbgpt.serve.agent.team.layout.team_awel_layout import AwelLayoutChatManger
from dbgpt.agent.agents.expand.plugin_assistant_agent import PluginAssistantAgent
from dbgpt.agent.agents.expand.summary_assistant_agent import SummaryAssistantAgent

from dbgpt.agent.agents.agent import AgentContext
from dbgpt.agent.memory.gpts_memory import GptsMemory
from dbgpt.core.interface.llm import ModelMetadata

import asyncio

from dbgpt.model import OpenAILLMClient

In [7]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
test_plugin_dir = os.path.join(parent_dir, "test_files")

In [8]:
os.environ['OPENAI_API_KEY']="sk-x"
os.environ['OPENAI_API_BASE']="https://proxy_url/v1"
os.environ['BAIDU_COOKIE']="""your baidu cookie"""

llm_client = OpenAILLMClient()
context: AgentContext = AgentContext(conv_id="test456", llm_provider=llm_client)
context.llm_models = [ModelMetadata(model="gpt-3.5-turbo")]
context.gpts_name = "信息析助手"

default_memory = GptsMemory()
manager = AwelLayoutChatManger(
    agent_context=context,
    memory=default_memory,
)

### agents
tool_enginer = PluginAssistantAgent(
    agent_context=context,
    memory=default_memory,
    plugin_path=test_plugin_dir,
)
summarizer = SummaryAssistantAgent(
    agent_context=context,
    memory=default_memory,
)

manager.hire([tool_enginer, summarizer])

user_proxy = UserProxyAgent(memory=default_memory, agent_context=context)


await user_proxy.a_initiate_chat(
    recipient=manager,
    reviewer=user_proxy,
    message="查询成都今天天气",
)



User (to layout_manager)-[]:

"查询成都今天天气"

--------------------------------------------------------------------------------
layout_manager (to ToolScientist)-[]:

"查询成都今天天气"

--------------------------------------------------------------------------------
un_stream ai response: {
  "tool_name": "google_search",
  "args": {
    "query": "成都今天天气"
  },
  "thought": "I will use the google-search tool to search for the weather in Chengdu today."
}
{'query': '成都今天天气'}
_google_search:成都今天天气
ToolScientist (to Summarizer)-[gpt-3.5-turbo]:

"{\n  \"tool_name\": \"google_search\",\n  \"args\": {\n    \"query\": \"成都今天天气\"\n  },\n  \"thought\": \"I will use the google-search tool to search for the weather in Chengdu today.\"\n}"
>>>>>>>>ToolScientist Review info: 
 Pass.None
>>>>>>>>ToolScientist Action report: 
execution succeeded,
Error: Please configure GOOGLE_API_KEY and GOOGLE_API_CX in .env first!

--------------------------------------------------------------------------------
un_stream ai r